In [165]:
import pandas as pd
import sqlite3
db = sqlite3.connect('claims.db')

In [175]:
# This was to populate p1-15, d1-25 the cleaner and lazier way

def proc_diag(s,start, end): #this function is to help me populate the columns D1-D25, and P1-P15
    result = []

    for i in range(start, end+1):
        result.append(s + str(i))
    
    return result


p = proc_diag('P',1,15)
d = proc_diag('D',1,25)


def concat_columns(lst):
    result = ''
    for i in lst:
        result += str(i) + '||' 
    return result[:-2]


d_columns = concat_columns(d)

# this is to check every combo of p1-p15 and d1-d25
def long_condition(proc_diag, code_number): # can only use on specific diagnoses
    result = ""
    for i in proc_diag:
        for j in code_number:
            result += str(i)+ ' = \'' + str(j)+ '\'' + ' OR '
    return result

check_for_maglignant_colon = long_condition(d, ['152'])


#changes list to string
def list_to_string (lst):
    result = ''
    for i in lst:
        result += str(i) + ', ' 
    return result[:-2]

d_string = list_to_string(d)


pcs = [45.8,45.7,45.23]    
p_diagnose_code = long_condition(p, pcs) 




In [167]:
# this code works well if you only are checking for one item and it's a little more general
# in our assignment it says 152. and I could hard code 152.1, 152.2 but I wanted to try it a different way

def diagnosis (diagnosis_code) : 
    return pd.read_sql_query(
                            'SELECT encounter_key, patient_id, doctor_id, ' + d_columns + ''' as diagnoses
                              FROM medical_headers
                              WHERE diagnoses LIKE '%'''
                                + diagnosis_code + "%'",db)

In [168]:
#These people needed colonoscopies
colon_cancer = diagnosis('152.')

In [174]:
def multiple_diagnosis (list_diagnosis_code):
    return pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
                               FROM medical_headers
                              WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
                              NOT IN (SELECT patient_id
                              FROM medical_headers
                              WHERE ''' + d_columns + ''' LIKE '%152%')
                              ''', db)


# def multiple_diagnosis (list_diagnosis_code):
#     return pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
#                                FROM medical_headers
#                               WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
#                               NOT IN''' colon_cancer.patient_id
# This would be the better way to do it. check for the patient_id's in the magliment dataframe
# It the patient_id is there than they cannot be in this dataframe
  

In [170]:
begnin = long_condition(d, ['211.3', '211.4'])
#These people do NOT need colonoscopies this is my denomonator
not_colon_cancer = multiple_diagnosis(begnin)

In [177]:
not_colon_cancer.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9699 entries, 0 to 9698
Data columns (total 3 columns):
encounter_key    9699 non-null object
patient_id       9699 non-null object
doctor_id        9699 non-null object
dtypes: object(3)
memory usage: 227.4+ KB


In [194]:

def nums_between (start,end):
    result = []
    for i in range (start, end+1):
        result.append(i)
        
    return result

cpt = [45378, 45388, 44110 ] + nums_between(45380,45385) + nums_between (44150,44160) + nums_between(44204, 44208) + nums_between(44210, 44212) + [44146]
only_resection = [44110,44146] + nums_between (44150,44160) +nums_between(44204, 44208) + nums_between(44210, 44212)

In [195]:
cpt_string = list_to_string(cpt)
resection_string = list_to_string(only_resection)

In [184]:
cpt_string

'45378, 45388, 44110, 45380, 45381, 45382, 45383, 45384, 45385, 44150, 44151, 44152, 44153, 44154, 44155, 44156, 44157, 44158, 44159, 44160, 44204, 44205, 44206, 44207, 44208, 44210, 44211, 44212, 44146'

In [187]:
#Colonoscopy/colectomy was preformed, this is my numerator

cc = pd.read_sql_query('''SELECT encounter_key, procedure
                               FROM medical_service_lines
                              WHERE procedure in (''' + cpt_string + ''')''', db)


In [196]:
#This is for only those who got a colectomy

resection = pd.read_sql_query('''SELECT encounter_key, procedure
                               FROM medical_service_lines
                              WHERE procedure in (''' + resection_string+ ''')''', db)


In [189]:

fp= not_colon_cancer.merge(cc, left_on = not_colon_cancer.encounter_key, right_on= cc.encounter_key)
fp.drop(['encounter_key_x', 'encounter_key_y'], axis = 1)

,key_0,patient_id,doctor_id,procedure
0,b2ff5e25ef396d1b37da4bd7f7bc4ed7bf63242f6807de...,dce553a9508c25e439171274397adf3be0c31182c5d4bf...,924f908412a2243aefaaf1a1abbe0651e668e6340c3993...,45380
1,b2ff5e25ef396d1b37da4bd7f7bc4ed7bf63242f6807de...,dce553a9508c25e439171274397adf3be0c31182c5d4bf...,924f908412a2243aefaaf1a1abbe0651e668e6340c3993...,45385
2,19b516dd184c38e369d34fc774286bcc702b16759753f3...,4544ac27e14e02f16689c6ae2dab0a004b1610617e4b11...,95558115a97a04dea62cc868bc5a4f7868c59771eb8376...,45385
3,3fb27b882f9ba1da80b2b8fe68c436ceb572c5d69abc5e...,09ba0c67ea0d3e24fc368cf371f022c2cb18f2a3caa48b...,f4fe9d68e9a06488e63ab414c44bbb6f7c37ecb6a3320c...,45385
4,3fb27b882f9ba1da80b2b8fe68c436ceb572c5d69abc5e...,09ba0c67ea0d3e24fc368cf371f022c2cb18f2a3caa48b...,f4fe9d68e9a06488e63ab414c44bbb6f7c37ecb6a3320c...,45380
5,71a18e016c8f8eeb7111ad7159617b345255d4935dcd91...,d401258d207d2cf3293bfaf7a995e840cc6573fc4a2fc3...,78b02a2c050ea90125bf371b4cde1130f6880b77367eeb...,44152
6,71a18e016c8f8eeb7111ad7159617b345255d4935dcd91...,d401258d207d2cf3293bfaf7a995e840cc6573fc4a2fc3...,78b02a2c050ea90125bf371b4cde1130f6880b77367eeb...,44146
7,bb0d2a8f5fdd024ea661779870e7f7561711b740acfaeb...,6a8ce61568f62db2345aada7a4a1bdbdb7c1384251709d...,6b1f58cd5db235b4cd16a545fca474def0e81936128329...,45380
8,dfeccd5dfa60e9e16c8da8b55857e99e73353b265ad4da...,3c076dee7a10772565366e632326f01f3cd15d40d08f8c...,d862fe4831da07a0dd897cf1e598756e82c0f3b457c81b...,45385
9,c1f7f8643a656b095f05e58e2f1b84f18dfe9c85814c85...,6049f1f9786d6568eaa9306b4e87c6bae7e1ebc55f69d9...,9a96f55a48834babe711879d83f19da8f33a52feab1d17...,44150


In [191]:
result = pd.merge(not_colon_cancer, cc, how="left", on=["encounter_key"])


In [202]:
ben_got_col =  pd.merge(not_colon_cancer, resection, how="left", on=["encounter_key"])

KeyError: 'patient_id'

In [193]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12887 entries, 0 to 12886
Data columns (total 4 columns):
encounter_key    12887 non-null object
patient_id       12887 non-null object
doctor_id        12887 non-null object
procedure        11858 non-null object
dtypes: object(4)
memory usage: 503.4+ KB


In [199]:
#out of the 12887 people who were benign 11858 got a colonoscopy or resection and 2173 got colonectomy


ben_got_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10038 entries, 0 to 10037
Data columns (total 4 columns):
encounter_key    10038 non-null object
patient_id       10038 non-null object
doctor_id        10038 non-null object
procedure        2174 non-null object
dtypes: object(4)
memory usage: 392.1+ KB


In [208]:
# not_colon_cancer['patients'] = not_colon_cancer.groupby(['doctor_id'])
#not enough time but i was going to use the groupby from pandas library to gather how many patients the doctor had
#how many were bengin without any procedures and how many had procedures despite being bengin

In [206]:
ben_got_col

,encounter_key,patient_id,doctor_id,procedure
0,b2ff5e25ef396d1b37da4bd7f7bc4ed7bf63242f6807de...,dce553a9508c25e439171274397adf3be0c31182c5d4bf...,924f908412a2243aefaaf1a1abbe0651e668e6340c3993...,NaN
1,ecff758b15a1a6e93c09f6d3adc7033b27ba48f824d045...,f157360c6b7894973d598204e725a5887b27b0072f2d13...,880251f5ae409dc658535736b41eadc8456f569f7f9828...,NaN
2,19b516dd184c38e369d34fc774286bcc702b16759753f3...,4544ac27e14e02f16689c6ae2dab0a004b1610617e4b11...,95558115a97a04dea62cc868bc5a4f7868c59771eb8376...,NaN
3,4c231bb6dc7e12b6b54af0ca6d421b3203d2cf4f61f923...,f9f5840f724cd2b0007f49b5d8a10eec5b25da82f5d605...,9de9d46a7b57ce0ad6c4d6e26780c8b4a49252c0957409...,NaN
4,3fb27b882f9ba1da80b2b8fe68c436ceb572c5d69abc5e...,09ba0c67ea0d3e24fc368cf371f022c2cb18f2a3caa48b...,f4fe9d68e9a06488e63ab414c44bbb6f7c37ecb6a3320c...,NaN
5,71a18e016c8f8eeb7111ad7159617b345255d4935dcd91...,d401258d207d2cf3293bfaf7a995e840cc6573fc4a2fc3...,78b02a2c050ea90125bf371b4cde1130f6880b77367eeb...,44152
6,71a18e016c8f8eeb7111ad7159617b345255d4935dcd91...,d401258d207d2cf3293bfaf7a995e840cc6573fc4a2fc3...,78b02a2c050ea90125bf371b4cde1130f6880b77367eeb...,44146
7,bb0d2a8f5fdd024ea661779870e7f7561711b740acfaeb...,6a8ce61568f62db2345aada7a4a1bdbdb7c1384251709d...,6b1f58cd5db235b4cd16a545fca474def0e81936128329...,NaN
8,dfeccd5dfa60e9e16c8da8b55857e99e73353b265ad4da...,3c076dee7a10772565366e632326f01f3cd15d40d08f8c...,d862fe4831da07a0dd897cf1e598756e82c0f3b457c81b...,NaN
9,c1f7f8643a656b095f05e58e2f1b84f18dfe9c85814c85...,6049f1f9786d6568eaa9306b4e87c6bae7e1ebc55f69d9...,9a96f55a48834babe711879d83f19da8f33a52feab1d17...,44150
